In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


# Clients

In [2]:
# For "registered" clients (via `@Client.register`), the client
# can be created with `create_client` by passing in the model name.
from sik_llms import create_client

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [3]:
# Or, the client can be directly instantiated
from sik_llms import OpenAI
client = OpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [4]:
# Or, the client can be directly instantiated
from sik_llms import Anthropic
client = Anthropic(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
client

# Chat

In [3]:
from sik_llms import create_client, user_message, TextChunkEvent

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously via `__call__`

In [7]:
response = client(messages=[message])
print(response)
print(response.response)

input_tokens=14 output_tokens=7 input_cost=2.1e-06 output_cost=4.2e-06 duration_seconds=0.728931188583374 response='The capital of France is Paris.'
The capital of France is Paris.


### Run Aynchronously via `run_async`

In [13]:
response = await client.run_async(messages=[message])
print(response)
print(response.response)

input_tokens=14 output_tokens=7 input_cost=2.1e-06 output_cost=4.2e-06 duration_seconds=0.5546269416809082 response='The capital of France is Paris.'
The capital of France is Paris.


### Stream Asynchronously via `stream`

In [11]:
responses = []
async for response in client.stream(messages=[message]):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    responses.append(response)

The capital of France is Paris.

In [12]:
responses

[TextChunkEvent(content='The', logprob=0.0),
 TextChunkEvent(content=' capital', logprob=0.0),
 TextChunkEvent(content=' of', logprob=0.0),
 TextChunkEvent(content=' France', logprob=0.0),
 TextChunkEvent(content=' is', logprob=0.0),
 TextChunkEvent(content=' Paris', logprob=-1.1160349458805285e-05),
 TextChunkEvent(content='.', logprob=-6.704273118884885e-07),
 TextResponse(input_tokens=14, output_tokens=7, input_cost=2.1e-06, output_cost=4.2e-06, duration_seconds=0.572174072265625, response='The capital of France is Paris.')]

### Claude 

In [9]:
client = create_client(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
response = client(messages=[user_message("What is the capital of France?")])
response

ResponseSummary(input_tokens=14, output_tokens=49, input_cost=4.2000000000000004e-05, output_cost=0.000735, duration_seconds=1.1158390045166016, response="The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.")

# OpenAI Functions/Tools

In [2]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type=str,
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_TOOLS,
    model_name='gpt-4o-mini',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=60 output_tokens=17 input_cost=9e-06 output_cost=1.0199999999999999e-05 duration_seconds=0.7561488151550293 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_PUeEPiBRUbF3aGFUfq0NyU7u') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='call_PUeEPiBRUbF3aGFUfq0NyU7u'


---

# Claude Functions/Tools

In [1]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type=str,
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_TOOLS,
    model_name='claude-3-7-sonnet-latest',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=401 output_tokens=40 input_cost=0.001203 output_cost=0.0006000000000000001 duration_seconds=1.515820026397705 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_01PNN6At5XHFmYTerwTabuh5') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='toolu_01PNN6At5XHFmYTerwTabuh5'


---

# Structured Outputs via OpenAI

In [ ]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = await client.run_async(messages=messages)
# or `response = client(messages=messages)` for synchronous execution
print(response)
print('---')
print(response.parsed)

input_tokens=92 output_tokens=18 input_cost=1.38e-05 output_cost=1.08e-05 duration_seconds=0.7941067218780518 parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']) refusal=None
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Structured Outputs via Anthropic

In [4]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = await client.run_async(messages=messages)
# or `response = client(messages=messages)` for synchronous execution
print(response)
print('---')
print(response.parsed)

input_tokens=441 output_tokens=78 input_cost=0.001323 output_cost=0.00117 duration_seconds=2.1678481101989746 parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']) refusal=None
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Reasoning via OpenAI

In [ ]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, TextResponse, ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.stream(messages=messages):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    elif isinstance(response, TextResponse):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

Let's break it down step by step:

1. Calculate inside the parentheses:
   - 3 * 4 = 12
   - 5 * 6 = 30

2. Replace the original expression with these values:
   1 + 2 + 12 + 30

3. Add them together:
   1 + 2 = 3
   3 + 12 = 15
   15 + 30 = 45

So, the answer is 45.

In [16]:
summary

ResponseSummary(input_tokens=28, output_tokens=104, input_cost=3.08e-05, output_cost=0.0004576, duration_seconds=2.3381471633911133, response="Let's break it down step by step:\n\n1. Calculate inside the parentheses:\n   - 3 * 4 = 12\n   - 5 * 6 = 30\n\n2. Replace the original expression with these values:\n   1 + 2 + 12 + 30\n\n3. Add them together:\n   1 + 2 = 3\n   3 + 12 = 15\n   15 + 30 = 45\n\nSo, the answer is 45.")

---

# Reasoning via Claude

In [ ]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, ThinkingChunkEvent,
    TextResponse, ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None

current_type = None
async for response in client.stream(messages=messages):
    is_text_chunk = isinstance(response, TextChunkEvent)
    is_thinking_chunk = isinstance(response, ThinkingChunkEvent)
    is_summary = isinstance(response, TextResponse)

    if is_text_chunk or is_thinking_chunk:
        if type(response) is not current_type:
            print(f"\n\n[{'THINKING' if is_thinking_chunk else 'TEXT'}]")
            current_type = type(response)
        print(response.content, end="")
    elif isinstance(response, TextResponse):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")



[THINKING]
Let me solve this step by step, following the order of operations (PEMDAS).

First, let's calculate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Now I can rewrite the expression as:
1 + 2 + 12 + 30

Now I'll add these numbers:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

So the final answer is 45.

[TEXT]
To solve this expression, I'll use the order of operations (PEMDAS):

First, I'll evaluate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Now I can rewrite the expression as:
1 + 2 + 12 + 30

Adding these numbers:
1 + 2 + 12 + 30 = 45

The answer is 45.

In [18]:
summary

ResponseSummary(input_tokens=60, output_tokens=248, input_cost=0.00018, output_cost=0.00372, duration_seconds=4.046967029571533, response="Let me solve this step by step, following the order of operations (PEMDAS).\n\nFirst, let's calculate the expressions in parentheses:\n- (3 * 4) = 12\n- (5 * 6) = 30\n\nNow I can rewrite the expression as:\n1 + 2 + 12 + 30\n\nNow I'll add these numbers:\n1 + 2 = 3\n3 + 12 = 15\n15 + 30 = 45\n\nSo the final answer is 45.To solve this expression, I'll use the order of operations (PEMDAS):\n\nFirst, I'll evaluate the expressions in parentheses:\n- (3 * 4) = 12\n- (5 * 6) = 30\n\nNow I can rewrite the expression as:\n1 + 2 + 12 + 30\n\nAdding these numbers:\n1 + 2 + 12 + 30 = 45\n\nThe answer is 45.")

---

# ReasoningAgent

In [ ]:
import json
from sik_llms.models_base import (
    Tool, Parameter, ThinkingEvent, ToolPredictionEvent,
    ToolResultEvent, TextChunkEvent, ErrorEvent, TextResponse,
)
from sik_llms.reasoning_agent import ReasoningAgent

####
# Define the tool functions
####
async def calculator(expression: str) -> str:
    """Execute calculator tool."""
    try:
        # Only allow simple arithmetic for safety
        allowed_chars = set('0123456789+-*/() .')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e!s}"


async def weather(location: str, units: str) -> str:
    """Mock weather tool - returns fake data."""
    # Return mock weather data
    weather_data = {
        'New York': '68',
        'San Francisco': '62',
        'Miami': '85',
        'Chicago': '55',
        'Los Angeles': '75',
    }
    for city in weather_data:  # noqa: PLC0206
        if city.lower() in location.lower():
            temp = weather_data[city]
            if units == 'C':
                # C = (°F - 32) x (5/9)
                temp = round((temp - 32) * 5 / 9)
            return {location: f"{temp}°{units}"}
    return None

####
# Define tool objects
####
calculator_tool = Tool(
    name='calculator',
    description="Perform mathematical calculations",
    parameters=[
        Parameter(
            name='expression',
            param_type=str,
            required=True,
            description="The mathematical expression to evaluate (e.g., '2 + 2', '5 * 10')",
        ),
    ],
    func=calculator,
)

weather_tool = Tool(
    name="get_weather",
    description="Get the current weather for a location",
    parameters=[
        Parameter(
            name="location",
            param_type=str,
            required=True,
            description="The name of the city (e.g., 'San Francisco', 'New York', 'London')",
        ),
        Parameter(
            name='units',
            param_type=str,
            required=True,
            description="The units for temperature",
            valid_values=['F', 'C'],
        ),
    ],
    func=weather,
)

# Create the reasoning agent
agent = ReasoningAgent(
    model_name="gpt-4o-mini",  # You can change this to other models
    tools=[calculator_tool, weather_tool],
    max_iterations=10,
    temperature=0,
)

question = "I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?"  # noqa: E501
# Run the agent and collect the results
messages = [{"role": "user", "content": question}]

print(f"[QUESTION]: {question}\n")

current_iteration = 0

async for event in agent.stream(messages):
    if isinstance(event, ThinkingEvent):
        if hasattr(event, 'iteration') and event.iteration != current_iteration:
            current_iteration = event.iteration
            print(f"\n--- Iteration {current_iteration}\n")
        if event.content:
            print(f"\n[THINKING]:\n{event.content}")

    elif isinstance(event, ToolPredictionEvent):
        print("\n[TOOL PREDICTION]:")
        print(f"Tool: {event.name}`")
        print(f"Parameters: \n```json\n{json.dumps(event.arguments, indent=2)}\n```")

    elif isinstance(event, ToolResultEvent):
        print("\n[TOOL RESULT]:")
        print(f"Tool: {event.name}`")
        print(f"Result: {event.result}")

    elif isinstance(event, ErrorEvent):
        print("\n[ERROR]:")
        print(f"Error: {event.content}")

    elif isinstance(event, TextChunkEvent):
        # For streaming text generation
        if current_iteration  >= 0:  # Only print once for the header
            print("\n[FINAL RESPONSE]:")
            current_iteration = -1  # Set to an impossible iteration to avoid repeating

        print(event.content, end="")

    elif isinstance(event, TextResponse):
        # Print nothing here as we've already streamed the response
        pass

# Just for notebook display cleanliness
print("\n\n---\n\n")

[QUESTION]: I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?


--- Iteration 1


[THINKING]:
To provide the weather information for New York and Miami, I need to gather the current weather data for both cities. Additionally, I need to calculate the daily budget for a 6-day trip with a total budget of $2400. First, I will calculate the daily budget, and then I will check the weather for both cities.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "New York",
  "units": "F"
}
```

[TOOL RESULT]:
Tool: get_weather`
Result: {'New York': '68°F'}

--- Iteration 2


[THINKING]:
I have obtained the current weather for New York, which is 68°F. Now, I will proceed to get the weather for Miami. After that, I will calculate the daily budget for the trip.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "Miami",
  "units": "F"
}
`

In [23]:
print(f"Input Tokens: {event.input_tokens}")
print(f"Output Tokens: {event.output_tokens}")
print(f"Total Cost: {event.total_cost}")
print(f"Duration: {event.duration_seconds:.2f} seconds")

Input Tokens: 4924
Output Tokens: 512
Total Cost: 0.0010458
Duration: 11.69 seconds


---

# Prompt Caching

## Anthropic

In [4]:
from faker import Faker

cache_content = Faker().text(max_nb_chars=15_000)
cache_content[0:200]

'Difference tell behind travel attorney. Second provide eye term wife man. Business nothing person my stop listen.\nAct happy many imagine arm quality drug. Wear yard answer morning rest think art natio'

In [5]:
####
# This example is modified from anthropic's prompt-caching.ipynb notebook
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
####
from sik_llms import Anthropic
from sik_llms.models_base import system_message, user_message

client = Anthropic(
    model_name='claude-3-5-haiku-latest',
    temperature=0.1,
)
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
system_messages = [
    system_message("You are a helpful assistant."),
    system_message(
        cache_content,
        cache_control={'type': 'ephemeral'},
    ),
]
messages = [
    *system_messages,
    user_message("What is the first word of the cached text?"),
]

# first run should result in a cache-miss & write
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The first word of the cached text is "Difference".
---
Total Cost: 0.0029365999999999997
---
Input Tokens: 17
Output Tokens: 15
Cache Write Tokens: 2863
Cache Read Tokens: 0
Total Tokens: 2895


In [6]:
# second run should result in a cache-hit & read
messages = [
    *system_messages,
    user_message("What is the second word of the cached text?"),
]
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The second word of the cached text is "tell".
---
Total Cost: 0.00029864000000000003
---
Input Tokens: 17
Output Tokens: 14
Cache Write Tokens: 0
Cache Read Tokens: 2863
Total Tokens: 2894


---

# Misc Examples

## Bedrock via OpenAI API

In [3]:
import os
from dotenv import load_dotenv
from sik_llms import OpenAI, user_message
load_dotenv()

client = OpenAI(
    server_url=os.getenv('BEDROCK_API_URL'),
    api_key=os.getenv('BEDROCK_API_KEY'),
    model_name='anthropic.claude-3-haiku-20240307-v1:0',
    user='bedrock-requires-user?',
)
response = client(messages=[user_message("What is the capital of France?")])
print(response.response)

The capital of France is Paris.


---

## OpenAI Log Probs

In [ ]:
import math
from dotenv import load_dotenv
from sik_llms import OpenAI, user_message, TextChunkEvent
load_dotenv()

client = OpenAI(
    model_name='gpt-4o-mini',
    logprobs=True,
)
messages = [
    user_message("Write a haiku about the ocean."),
]
async for response in client.stream(messages=messages):
    if isinstance(response, TextChunkEvent):
        log_prob = response.logprob
        prob = math.exp(log_prob)
        print(f"{response.content} (logprob: {log_prob:.2f}, prob: {prob:.2f})")

Wh (logprob: -1.76, prob: 0.17)
ispers (logprob: -0.00, prob: 1.00)
 of (logprob: -0.10, prob: 0.91)
 the (logprob: -0.00, prob: 1.00)
 tide (logprob: -0.38, prob: 0.68)
, (logprob: -0.00, prob: 1.00)
  
 (logprob: -0.00, prob: 1.00)
S (logprob: -7.07, prob: 0.00)
apphire (logprob: -2.21, prob: 0.11)
 waves (logprob: -0.08, prob: 0.93)
 kiss (logprob: -0.38, prob: 0.68)
 sandy (logprob: -1.69, prob: 0.18)
 shore (logprob: -2.58, prob: 0.08)
, (logprob: -0.01, prob: 0.99)
  
 (logprob: -0.00, prob: 1.00)
Secrets (logprob: -0.49, prob: 0.61)
 deep (logprob: -1.15, prob: 0.32)
 reside (logprob: -0.60, prob: 0.55)
. (logprob: -0.00, prob: 1.00)
   (logprob: -1.70, prob: 0.18)
